In [1]:
import os
from utils import *
import pickle

In [2]:
def oracle(num_bit,dim_query,dim_work):
    # construct the orcale for all x
    # num_bit : number of bit for boolean function
    # dim_query : dimension of query register
    # dim_work : dimension of working memory register
    tmp0 = np.arange(2**num_bit, dtype=np.uint64).reshape(-1,1)
    x_bit = unpackbits(tmp0, axis=-1, count=dim_query, bitorder='l').T
    oracle = np.zeros([dim_acc,2**num_bit],dtype=np.complex128)
    for i in range(2**num_bit):
        oracle[:,i] = np.kron((-1)**x_bit[:,i],np.ones(dim_work))
    return oracle

def forward(dim_acc,num_bit,unitary,oracle):
    state = np.zeros([dim_acc,2**num_bit],dtype=np.complex128)
    state[0] = 1
    state = unitary[0]@state
    for i in range(1,len(unitary)):
        state = oracle*state
        state = unitary[i]@state
    return state

In [3]:
file_list = os.listdir("data/")
for file in file_list:
    with open("data/"+file,"rb") as f:
        data= pickle.load(f)
    n,k,l = data["n"],data["k"],data["l"]
    dim_query = n+1
    partition = data["partition"]
    dim_acc = sum(partition)
    dim_work = dim_acc//dim_query
    weight = data["f(x)"]
    num_query = len(data["unitary_list"])-1
    state = forward(dim_acc,n,data["unitary_list"],oracle(n,dim_query,dim_work))
    mes = mes_matrix(partition,weight)
    error_rate = np.max(1-np.sum(np.abs(state*mes)**2,axis=0))
    print(f"EXACT({n},{k},{l})\t {num_query}query error rate:{error_rate}")

EXACT(12,4,8)	 7query error rate:2.969999270963086e-07
EXACT(11,9,10)	 9query error rate:6.747278391561196e-08
EXACT(16,2,14)	 13query error rate:6.461945106783773e-07
EXACT(13,11,12)	 11query error rate:7.841088556226339e-09
EXACT(11,6,7)	 7query error rate:2.5745128430232356e-06
EXACT(13,12,13)	 12query error rate:9.181477800268567e-10
EXACT(14,3,11)	 10query error rate:1.0550785692453601e-05
EXACT(7,5,6)	 5query error rate:8.133358431194893e-10
EXACT(11,8,9)	 8query error rate:9.849659055038984e-08
EXACT(7,6,7)	 6query error rate:1.6397994073713562e-12
EXACT(14,4,10)	 9query error rate:5.981798704146613e-07
EXACT(13,10,11)	 10query error rate:3.046532970008009e-06
EXACT(14,2,12)	 11query error rate:4.004814641911381e-06
EXACT(10,4,6)	 5query error rate:7.768092131765769e-07
EXACT(10,3,7)	 6query error rate:4.972727912555541e-06
EXACT(12,2,10)	 9query error rate:6.5077902923205144e-06
EXACT(9,7,8)	 7query error rate:8.026038603725638e-07
EXACT(11,7,8)	 7query error rate:6.10205248641